# Keras example

Trains and evaluate a simple MLP on the Reuters newswire topic classification task.

In [1]:
import numpy as np

import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer

# The following import and function call are the only additions to code required
# to automatically log metrics and parameters to MLflow.
import mlflow.keras

Using TensorFlow backend.


In [2]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0
remote_server_uri = "http://0.0.0.0:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

exp_name = "reuters_keras"
mlflow.set_experiment(exp_name)

INFO: 'reuters_keras' does not exist. Creating a new experiment


In [3]:
mlflow.keras.autolog()

In [10]:
max_words = 1000
batch_size = 32
epochs = 5

mlflow.log_params({"max_words": max_words, "batch_size": batch_size, "epochs": epochs})

In [5]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

Loading data...
8982 train sequences
2246 test sequences
46 classes


In [6]:
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Vectorizing sequence data...
x_train shape: (8982, 1000)
x_test shape: (2246, 1000)


In [7]:
print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (8982, 46)
y_test shape: (2246, 46)


In [8]:
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Building model...


In [9]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 8083 samples, validate on 899 samples
Epoch 1/5
8083/8083 [==============================] - 7s 908us/step - loss: 1.4220 - accuracy: 0.6785 - val_loss: 1.0903 - val_accuracy: 0.7597
Epoch 2/5
8083/8083 [==============================] - 3s 372us/step - loss: 0.7866 - accuracy: 0.8129 - val_loss: 0.9159 - val_accuracy: 0.7931
Epoch 3/5
8083/8083 [==============================] - 7s 900us/step - loss: 0.5530 - accuracy: 0.8666 - val_loss: 0.8490 - val_accuracy: 0.8109
Epoch 4/5
8083/8083 [==============================] - 5s 658us/step - loss: 0.4166 - accuracy: 0.8969 - val_loss: 0.8406 - val_accuracy: 0.8087
Epoch 5/5
8083/8083 [==============================] - 4s 484us/step - loss: 0.3240 - accuracy: 0.9170 - val_loss: 0.8777 - val_accuracy: 0.8087


In [21]:
# what is missing in the log? 

In [12]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])
mlflow.log_metrics({"test_accuarcy": score[1], "test_loss": score[0]})

2246/2246 [==============================] - 1s 243us/step
Test score: 0.8933801099007829
Test accuracy: 0.7947462201118469
